In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import accuracy_score

In [ ]:
df = yf.download('GOOGL', start='2010-01-01', end='2024-01-01')

In [ ]:
df.head()

In [ ]:
df = df[['Close']]

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df)

In [ ]:
def create_dataset(data, time_step=60):
    X, y = [], []
    for i in range(time_step, len(data)):
        X.append(data[i-time_step:i, 0])
        y.append(1 if data[i, 0] > data[i-1, 0] else 0) # 1 for increase, 0
for decrease
    return np.array(X), np.array(y)

In [ ]:
X, y = create_dataset(scaled_data)
X = X.reshape(X.shape[0], X.shape[1], 1) # Reshape for RNN [samples,
time_steps, features]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
random_state=42)

In [ ]:
model = Sequential()

In [ ]:
model.add(LSTM(units=50, return_sequences=True,
input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))

In [ ]:
model.add(Dense(units=1, activation='sigmoid')) # Binary output: 1
(increase), 0 (decrease)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy',
metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32,
validation_data=(X_test, y_test))

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

In [ ]:
last_60_days = scaled_data[-60:]
last_60_days = last_60_days.reshape(1, -1)
last_60_days = last_60_days.reshape((1, 60, 1))

In [ ]:
predicted_trend = model.predict(last_60_days)
predicted_trend = 1 if predicted_trend[0][0] > 0.5 else 0

In [ ]:
if predicted_trend == 1:
    print("The stock price is predicted to increase tomorrow.")
else:
    print("The stock price is predicted to decrease tomorrow.")

In [ ]:
predicted_stock_price = model.predict(X_test)
predicted_stock_price = (predicted_stock_price > 0.5) # Convert to 0 or 1
plt.figure(figsize=(10, 6))
plt.plot(y_test, color='blue', label='Real Stock Price Trend')
plt.plot(predicted_stock_price, color='red', label='Predicted Stock Price
Trend')
plt.title('Stock Price Trend Prediction')
plt.xlabel('Days')
plt.ylabel('Trend (1 = Increase, 0 = Decrease)')
plt.legend()
plt.show()